## Obtaining results

In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import os 
while 'notebooks' in os.getcwd():
    os.chdir('..')

import numpy as np
from src.simulations import PathSimulator
from src.simulation_based_tree import SimulationBasedModel

from time import time

In [21]:
kappa = 5.0
K = 10
r = 0.1
eta = 0.9
theta = 0.16
rho = 0.1
T = 0.25
v0 = 0.25**2
# S0 = 10
M = 10_000

In [22]:
simulator = PathSimulator()
pricer = SimulationBasedModel()

In [30]:
[3] + [1,2,3]

[3, 1, 2, 3]

In [23]:
reference_values = {
    8: 2,
    9: 1.1076,
    10: 0.52,
    11: 0.2137,
    12: 0.0820

}

In [28]:
for n in [100, 200, 500]:

    print(f"\n-------------------- N = {n} -------------------------\n")

    for S0 in [8,9,10,11,12]:

        St, vt = simulator.heston(
            T,
            n*1000,
            n,
            rho,
            kappa,
            theta,
            v0,
            S0,
            eta,
            r
        )

        g = lambda x, k : np.maximum(k-x, 0) # put option

        t0 = time()
        option_price_counting = pricer.compute_option_prices_counting(St, vt, g, n, T, K, r, v0)
        t1 = time()
        option_price_probabilities = pricer.compute_option_prices_probabilities(St, vt, g, n, T, K, r,v0)
        t2 = time()

        print(f"S0 = {S0}. Reference value: {reference_values[S0]}. Counting approach: {round(option_price_counting, 4)} ({round(t1-t0, 2)} s). Probabilities approach: {round(option_price_probabilities, 4)} ({round(t2-t1, 2)} s)")


-------------------- N = 100 -------------------------

S0 = 8. Reference value: 2. Counting approach: 2.0 (5.95 s). Probabilities approach: 2.0814 (4.47 s)
S0 = 9. Reference value: 1.1076. Counting approach: 1.0911 (5.95 s). Probabilities approach: 1.171 (4.48 s)
S0 = 10. Reference value: 0.52. Counting approach: 0.5066 (6.0 s). Probabilities approach: 0.5377 (4.45 s)
S0 = 11. Reference value: 0.2137. Counting approach: 0.2006 (5.94 s). Probabilities approach: 0.203 (4.48 s)
S0 = 12. Reference value: 0.082. Counting approach: 0.0712 (5.92 s). Probabilities approach: 0.0611 (4.41 s)

-------------------- N = 200 -------------------------

S0 = 8. Reference value: 2. Counting approach: 2.0 (24.0 s). Probabilities approach: 2.062 (20.27 s)
S0 = 9. Reference value: 1.1076. Counting approach: 1.1 (24.55 s). Probabilities approach: 1.1531 (19.88 s)
S0 = 10. Reference value: 0.52. Counting approach: 0.5144 (24.5 s). Probabilities approach: 0.5368 (19.86 s)
S0 = 11. Reference value: 0.2137. 